In [ ]:
from semantic_kernel import __version__
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

__version__

In [ ]:
from datetime import datetime
import os
now = datetime.now()
output_dir = f"./output/{now.strftime('%Y_%m_%d_%H_%M_%S')}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from semantic_kernel import Kernel

kernel = Kernel()

In [ ]:
from src.services import Service

from src.service_settings import ServiceSettings

service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    #Service.AzureOpenAI
    Service.OpenAI
    #if service_settings.global_llm_service is None
    #else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {service_settings.global_llm_service.lower()} {selectedService}")

In [ ]:
from openai import AsyncAzureOpenAI, AsyncOpenAI, OpenAI
from src.image_gen_text_to_image import ImageGenTextToImage

# image_client = AsyncAzureOpenAI(
#     azure_endpoint=os.getenv("AZURE_IMAGE_GEN_ENDPOINT"),
#     api_key=os.getenv("AZURE_IMAGE_GEN_API_KEY"),
#     api_version=os.getenv("AZURE_IMAGE_GEN_API_VERSION"),
# )
image_client = AsyncOpenAI()
sync_image_client = OpenAI()   

# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id="gpt-4.1"
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

from semantic_kernel.connectors.ai.open_ai import OpenAITextToImage 

image_gen_service = ImageGenTextToImage (
    async_client=image_client,
    ai_model_id="gpt-image-1",
    service_id="gpt-image-1", # Optional; for targeting specific services within Semantic Kernel
)
kernel.add_service(image_gen_service)

In [ ]:
import base64

# Function to convert a file to Base64
def file_to_base64(file_path):
    with open(file_path, "rb") as file:
        # Read the file in binary mode and encode it to Base64
        base64_encoded = base64.b64encode(file.read()).decode("utf-8")
    return base64_encoded

# Example usage
# file_path = "image.png"  # Replace with your file path
# base64_string = file_to_base64(file_path)


In [ ]:
image_gen_service = kernel.get_service(service_id="gpt-image-1")

### Generate the Idea

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
import time

seed = int(time.time())
print(f"Seed: {seed}")
execution_settings = OpenAIChatPromptExecutionSettings(seed=seed)

text_gen_service = kernel.get_service(service_id="default")

chat_history = ChatHistory()

input = "animal"

chat_history.add_message(ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text=f"""GENERATE A ONE-SENTENCE IDEA FOR A PRODUCT THAT CAN BE 3D PRINTED BY A RASIN 3D PRINTER.  
            IT SHOULD BE A USEFUL PRODUCT LIKE A TOOTHBRUSH HOLDER OR A HOOD ORNAMENT.  IT SHOULD BE FUNNY, CREATIVE AND WIMSICAL.

BE CREATIVE AND FUNNY. I WANT TO LAUGH.
Incorporate the style suggestion, if provided: {{$style}}
+++++

{input}
+++++
                                """),
#            ImageContent(uri=f"data:image/png;base64,{file_to_base64('./image.png')}")
       ]
    ))

# Get the chat completion response
idea = await text_gen_service.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

idea = str(idea)
print(idea)

### Generate Base Image

In [ ]:
print_idea = f"""Generate a photo of a plastic 3D-printed object based on the following idea: {idea}
The photo should be in a realistic style, with a white background and no text.
The object should be the only thing in the image, and it should be centered in the frame.
The object should be a 3D-printable object, and it should be in a realistic style.
The image must not be a drawing, painting, or cartoon.
"""

image = await image_gen_service.generate_image(
        description=print_idea, width=1024, height=1024, quality="auto"
    )

image_data = base64.b64decode(image)


base_image_path = f"{output_dir}/image.png"
with open(base_image_path, "wb") as image_file:
    image_file.write(image_data)


In [ ]:
from IPython.display import Image
Image(filename=base_image_path, width=300)

### Evaluate the image

In [ ]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

execution_settings = OpenAIChatPromptExecutionSettings()

text_gen_service = kernel.get_service(service_id="default")

chat_history = ChatHistory()

base64_string = file_to_base64(f"./{output_dir}/image.png")

# prompt = """Evaluate the following image from the point of view of marketability and ease of 3D printing.  
#                                 Make a one-sentense improvement suggestion.
#                                 Only respond with the suggestion.  Do not explain the reasons for the suggestion.
#                                 """

prompt = f"""I'm preparing this image for a 3D printing project. What objects should be removed from the image to make it more suitable for 3D printing?""" 

chat_history.add_message(ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text=prompt),
            ImageContent(uri=f"data:image/png;base64,{base64_string}")
        ]
    ))

# Get the chat completion response
eval_response = await text_gen_service.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

eval_response = str(eval_response)
print(eval_response)

### Update the Image

In [ ]:
import requests

def edit_image(image, mask, prompt):
    print("Editing image using OPENAI API...")
    result = sync_image_client.images.edit(
        model="gpt-image-1",
        image=[
            open(image, "rb"),
        ],
        prompt=prompt
    )
    print("Result:", result)

    return result.data[0].b64_json

def edit_image_azure(image, mask, prompt):
    url = f"{os.getenv('AZURE_IMAGE_GEN_ENDPOINT')}/openai/deployments/{os.getenv('AZURE_IMAGE_GEN_DEPLOYMENT_NAME')}/images/edits?api-version={os.getenv('AZURE_IMAGE_GEN_API_VERSION')}"

    headers = {
        "Authorization": f"Bearer {os.getenv('AZURE_IMAGE_GEN_API_KEY')}"
    }

    files = {
        'image': ('image.png', image),
        'mask': ('image.png', mask),
    }
    
    
    data = {
        "prompt": prompt
    }

    response = requests.post(url, data=data, files=files, headers=headers)

    print("Status Code:", response.status_code)
    with open("response.json", "w") as f:
        f.write(response.text)

    response_js = response.json()
    return response_js["data"][0]["b64_json"]



#image_base_64 = edit_image(open(f"{output_dir}/image.png", "rb").read(), open("mask.png", "rb").read(), eval_response)  
image_base_64 = edit_image(f"{output_dir}/image.png", "mask.png", "add a vase")  


image_data = base64.b64decode(image_base_64)


iter_image_path = f"{output_dir}/image2.png"
with open(iter_image_path, "wb") as image_file:
    image_file.write(image_data)

#img = Image.open(image_path)  # nosec
#img.show()


### Prelim. Results

In [ ]:
Image(filename=iter_image_path, width=300)

## Generate Marketing Material

In [ ]:
marketing1 = "Place this object on"

marks = ["a business desk", "a kitchen table", "a bathroom counter", "a living room table", "a coffee table", "a dining room table", "a shelf", "a bookcase", "a nightstand", "a dresser"]

i = -1
for mark in marks:
    i += 1
    m_image_path = f"{output_dir}/image2_{i}.png"
    if os.path.exists(m_image_path):
        print(f"File {m_image_path} already exists. Skipping...")
        continue
    prompt = f"{marketing1} {mark}."


    print(prompt)
    image_base_64 = edit_image(open(f"{output_dir}/image2.png", "rb").read(), open("mask2.png", "rb").read(), prompt)  


    image_data = base64.b64decode(image_base_64)


    
    with open(m_image_path, "wb") as image_file:
        image_file.write(image_data)
    
    display(Image(filename=m_image_path, width=300))

    

In [ ]:
!pip install matplotlib

In [45]:

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

base_dir = "C:\\Users\\antonslutsky\\Dev\\aml-examples\\3d_objects\\agents\\output\\20250525141109DUE"

# # Load the image
# image = cv2.imread(f'{base_dir}\\image_1.png', cv2.IMREAD_GRAYSCALE)

# # Apply thresholding to create a binary image
# _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# # Find contours
# contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# # Draw contours on the original image
# image_with_contours = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
# cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)

# # Display the result
# plt.imshow(cv2.cvtColor(image_with_contours, cv2.COLOR_BGR2RGB))
# plt.title('Contours')
# plt.axis('off')
# plt.show()

# # Create a directory to save the split images
# output_dir = f"{base_dir}/split_images"
# os.makedirs(output_dir, exist_ok=True)

# # Loop through each contour and save the corresponding region
# for i, contour in enumerate(contours):
#     # Get bounding box for each contour
#     x, y, w, h = cv2.boundingRect(contour)
    
#     # Extract the region of interest (ROI)
#     roi = image[y:y+h, x:x+w]
    
#     # Save the ROI as a separate image
#     output_path = os.path.join(output_dir, f"segment_{i}.png")
#     cv2.imwrite(output_path, roi)

# print(f"Images saved in '{output_dir}' directory.")

In [ ]:

import cv2
import numpy as np

# Load the image
image = cv2.imread(f'{base_dir}\\image_2.png')  # Replace 'image.jpg' with your image file path
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to create a binary image
_, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours and extract shapes
for contour in contours:
    # Approximate the contour to reduce the number of points
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Draw the contour on the original image
    cv2.drawContours(image, [approx], -1, (0, 255, 0), 3)

    # Identify the shape based on the number of vertices
    vertices = len(approx)
    if vertices == 3:
        shape = "Triangle"
    elif vertices == 4:
        shape = "Rectangle/Square"
    elif vertices > 4:
        shape = "Circle"
    else:
        shape = "Unknown"

    # Get the center of the shape
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv2.putText(image, shape, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Display the result
cv2.imshow('Shapes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
!pip install aspose-words

In [ ]:
from PIL import Image

# Open an image file
image = Image.open(f'{base_dir}\\image_2.png')

# Convert the image to grayscale
bw_image = image.convert("L")

# Save the black and white image
bw_image.save(f'{base_dir}\\image_2_gs.png')

# Optionally, show the image
bw_image.show()

In [62]:
# This code example demonstrates how to convert PNG to SVG
import aspose.words as aw

#  Create document object
doc = aw.Document()

# Create a document builder object
builder = aw.DocumentBuilder(doc)

# Load and insert PNG image
shape = builder.insert_image(f'{base_dir}\\image_blended.png')

# Specify image save format as SVG
saveOptions = aw.saving.ImageSaveOptions(aw.SaveFormat.SVG)

# Save image as SVG
shape.get_shape_renderer().save(f'{base_dir}\\image_blended.svg', saveOptions)

In [59]:
import sys
from PIL import Image
from potrace import Bitmap, POTRACE_TURNPOLICY_MINORITY  # `potracer` library

base_dir = ".\\output\\20250525174623jZQ"

def file_to_svg(filename: str):
    try:

        image = Image.open(filename)
    except IOError:
        print("Image (%s) could not be loaded." % filename)
        return
    bm = Bitmap(image, blacklevel=0.5)
    # bm.invert()
    plist = bm.trace(
        turdsize=2,
        turnpolicy=POTRACE_TURNPOLICY_MINORITY,
        alphamax=1,
        opticurve=False,
        opttolerance=0.2,
    )
    with open(f"{filename}.svg", "w") as fp:
        fp.write(
            f'''<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{image.width}" height="{image.height}" viewBox="0 0 {image.width} {image.height}">''')
        parts = []
        for curve in plist:
            fs = curve.start_point
            parts.append(f"M{fs.x},{fs.y}")
            for segment in curve.segments:
                if segment.is_corner:
                    a = segment.c
                    b = segment.end_point
                    parts.append(f"L{a.x},{a.y}L{b.x},{b.y}")
                else:
                    a = segment.c1
                    b = segment.c2
                    c = segment.end_point
                    parts.append(f"C{a.x},{a.y} {b.x},{b.y} {c.x},{c.y}")
            parts.append("z")
        fp.write(f'<path stroke="none" fill="black" fill-rule="evenodd" d="{"".join(parts)}"/>')
        fp.write("</svg>")

file_to_svg(f'{base_dir}\\image_blended.png')

generate a jigsaw puzzle of a magical castle.  only use black, red, green and blue colors for the castle.  Ensure that all the jigsaw pieces are clearly separated by a thick white border.

Generate a jigsaw puzzle of a magical castle.  
Only use black, red, green and blue colors for the castle.  
Ensure that all the jigsaw pieces are clearly separated by a thick white border.  
The puzzle must have 36 clearly visible pieces separated by a white border.

In [ ]:
import cv2

base_dir = ".\\output\\20250525174623jZQ"

img = cv2.imread(f'{base_dir}\\image.png')
mask = cv2.imread('puzzle_base_2.png',cv2.IMREAD_UNCHANGED)


# res = cv2.bitwise_and(img,img,mask = mask)
# res[mask==0] = 255

# cv2.imwrite(f'{base_dir}\\image_masked_2.png', res)

#cv2.imshow("puzzle", res)
try:
    from PIL import Image
except ImportError:
    import Image

background = Image.open(f'{base_dir}\\image.png')
overlay = Image.open('puzzle_base_masked_2.png')

background = background.convert("RGBA")
overlay = overlay.convert("RGBA")

new_img = Image.alpha_composite(background, overlay)

#new_img = Image.blend(background, overlay, 0.5)
new_img.save(f'{base_dir}\\image_blended_3.png',"PNG")

In [57]:
from PIL import Image

img = Image.open('puzzle_base_5.png')
rgba = img.convert("RGBA")
data = rgba.getdata()

target_color = (0, 0, 0, 255)

new_data = []


with open("data_out.txt", "w") as out:

    for item in data:
        # Replace the target color with transparency
        out.write(f"{item}\n")    
        if item[0] < 20 and item[1] < 20 and item[2] < 20:
            new_data.append((0, 0, 0, 0))  # Fully transparent
        else:
            new_data.append((255, 255, 255, 255))

    rgba.putdata(new_data)
    rgba.save("puzzle_base_masked_2.png", "PNG")

In [71]:
import cv2
import base64

# Load the image
image_path = './output/agent1/image1.png'

base64_bytes = base64.b64decode("iVBORw0KGgoAAAANSUhEUgAAAaQAAAGqCAIAAAC3LWgVAAAABmJLR0QA/wD/AP+gvaeTAAAACXBIWXMAAB7CAAAewgFu0HU+AACAAElEQVR4nOzde3hUZf7/8RdKTtqyQCIRoBERImQvGyyyDQsslGWTlQawWZKjLYlAkTOYRWfWOl2UlkmoiXZLiIjIXKbCl8IgYmTAJkUqhYkqNRm7N9VttZ2ISsoEBKehJSPX8//me/pEET+FXP1fz2effu/5+NjFz3+R36Aq9hARBFFFEEEUUUQj/IfjQy53+in395zLzcMplOgu6APD0AFrPzg+isAz99hRy8+2Bb14O//rJK3xn/0yS6DBPzjz2PoolDeA3gu4AYef9kxzHxP+Etus8dPoWvf7+SFD/Bn/CX7pwTh88++tTQAy8REUwShBFEJ88bd+Nb7n1Qi/E9IAHouYF+AOQykxET/aFLb0ONIH8f/VMDz3SeOL3uesz2IjrvLliNGJ+A/P4g2Pt7SUnSxe/NzU+AA//7IR02mnbR/nemocT8y/Z3BmBzz2Mhxf9LDG7ecamb+L5yvHeXH07CrNWvN42iLU8u/YPHfQc/+KuKH374keB9+sE2/Ey9M9jhD3Xx8imye/hRlTlIH3C08F+TvNRYNsOjE8sz9TvdfDPcuEM7PsC8VJgFgSnNfu/wnop/Niee7xhbhjLkInhoTB4rvenCOfNr54C+ZTseeC+F95qOM08L6ZWtHvbxuS754wxEn++GEts/cQFFYDonIrkWeu0/VYIzIhzc5HBigcopPOHvv337h1wl1V+AYZ8Jk60cGB9nw/j+PXabD7cp7870PDXD3kCrx7CfrpLdpq3frNC64K+dZz5wO4P/XqejQLqIS1LDRpBVUchk9PQDWsHPaZpHPETTZmKvHbT0cfBu59NNDx04TgXly08erC2nu9jM4NVNmDbittU4uiO+bDRXIhvA/tdY6Hsth4cR9htuCVBvzTJUD0fwqLYoernPPk+XShNOPpXj7oeQBePuDN+tXiigN9Zj2PaGXtEyvrGFRuha+N531koxju276oqSkOIQ+BgtYBUs9fhUToyGi+NjDbnqkufDHh08TPTy6y3omnQmCE959FXVK1XQi34Kq0onDQh9UBRGSKpEckNj1lcZ/eEup5Hlz45js9isRtvR3WtLHRMMNBgfuBQo9gi2aXb4IrP7mZDqORDKFPyeMnW5T8R8BlQc8c7ydI5su3Dpwoaq7HN3BoOW4b2FybCZgSBioE2BKAnYO+PH1kM4usxETCyqjlD2cEMFIma92zcmYaKrTVuamS60tsUh3CbCc8eqHWWOFyLGogPAScI2hhYfDQI3+t3MrAjx9mFETCOBRtxmTErk0ItCJDjZRQYHIwpDJnIVEtGad5pC5EumQAk9EIoHTxHTMcd+SonE2yvSUzGEDgF1k5jiPzdCo6Rdwe0WDzN4Oo9OVGksdy7mMjQQE0JlwoG/CLG2BroyTCFzrnI9or0DRbWw+jRQ6GSkgWmiUxHQThcxGoqUI8bha9h5HmozQs9CqAfS92LkG1O2ALNctUbUvCNTrSabAzCK7NijU38ckxJEZ93jkMfN2uhpphGhuMfKy+jKYZQP1ojiV6zUYUiQqxGkDawoctmtxBQwx7A/QSzBDskMtcdkRCowFyP3ahE2OCY8T8YmIxSimjUtsHQIoD9BLSIk2j5UFhme7hOljpplJMAAMu+IfNYyjMcR+ylgm4ytDEOGmttS+MaxT3IPxDjGSzVfpWhkQjiKkTKiPQJG9kc3KZKfwFCkA3RTB8qlrSlrHaC10IiCrTNaYcM1YN4BaYmEtojNGik2XeipUxTPeITqToTojsBwLQSaTRPaX+cpJX43MCADnKvhZUat11bSShEmiljZUfDjHkNFLUkTTqu66HZBRIciS0B11O9sMmSjdEfTqFkKzDpPoTAVn1MSiqOjDJlcNryQAPzRSvC8QmjV61CankHXbCEww3doq15I57PnIoSpDNFt9Jff7uc2D6cDqc7Now01ZZPWsqMmdEFC6paSgytZWxZ1ajK1BU1EPHFaTLxSCSOdSTJQR1v468zu8RW2kLAbytJCsC70i9FR3R2qnpzxujc9V4hwXxS8oiYEf8DW+i9ED/yAY95K6+5/mivy+3+uhcwvXC/8BG7xkXgOT5nhij4Z8Lf5ZZLA3Ez0ERmkQRRRBBFGk6gDQvwUiABFFFEEEUUYSwpkgSEEUUQQQi+xfiRcCABRRRBBFFEEkJ077r/AOS6WvtRltqgAAAABJRU5ErkJgg")

image = cv2.imread(base64_bytes)

# Display the image
cv2.imshow('Generated Tree Image', image)

# Wait for a key press and close the window
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Can't convert object to 'str' for 'filename'
>  - Can't convert object to 'str' for 'filename'
>  - Can't convert object to 'str' for 'filename'


In [72]:
from PIL import Image
import os

def extract_puzzle_pieces(image_path, rows, cols, output_dir):
    # Open the image
    img = Image.open(image_path)
    img_width, img_height = img.size

    # Calculate the size of each piece
    piece_width = img_width // cols
    piece_height = img_height // rows

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through grid and extract each piece
    for row in range(rows):
        for col in range(cols):
            # Define the bounding box for the current piece
            left = col * piece_width
            upper = row * piece_height
            right = left + piece_width
            lower = upper + piece_height

            # Crop the piece
            piece = img.crop((left, upper, right, lower))

            # Save the piece
            piece_filename = os.path.join(output_dir, f"piece_{row}_{col}.png")
            piece.save(piece_filename)

# Parameters for the puzzle
base_dir = "C:\\Users\\antonslutsky\\Dev\\aml-examples\\3d_objects\\agents\\output\\pro_202505291122"
image_path = f"{base_dir}\\image1.png"  # Path to the input image
output_dir = f"{base_dir}\\puzzle_pieces"  # Directory to save the pieces
rows, cols = 4, 4  # Number of rows and columns in the puzzle grid

extract_puzzle_pieces(image_path, rows, cols, output_dir)

In [1]:
from PIL import Image
import os

# Function to extract puzzle pieces
def extract_puzzle_pieces(image_path, rows, cols, output_dir):
    # Open the image
    img = Image.open(image_path)
    img_width, img_height = img.size

    # Calculate the size of each piece
    piece_width = img_width // cols
    piece_height = img_height // rows

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through grid and extract each piece
    for row in range(rows):
        for col in range(cols):
            # Define the bounding box for the current piece
            left = col * piece_width
            upper = row * piece_height
            right = left + piece_width
            lower = upper + piece_height

            # Crop the piece
            piece = img.crop((left, upper, right, lower))

            # Save the piece
            piece_filename = os.path.join(output_dir, f"piece_{row}_{col}.png")
            piece.save(piece_filename)

# Parameters for the puzzle
image_path = "./output/pro_202505291122/image1.png"  # Path to the input image
output_dir = "puzzle_pieces"  # Directory to save the pieces
rows, cols = 4, 4  # Number of rows and columns in the puzzle grid

In [7]:
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
import json
import pickle



chat_history = ChatHistory()

chat_history.add_message(ChatMessageContent(
        role=AuthorRole.USER,
        items=[TextContent(text="test")]
    ))

output = open('chat_history.pkl', 'wb')

pickle.dump(chat_history, output)

output.close()


In [8]:
pkl_file = open('chat_history.pkl', 'rb')

chat_history_loaded = pickle.load(pkl_file)
chat_history_loaded

ChatHistory(messages=[ChatMessageContent(inner_content=None, ai_model_id=None, metadata={}, content_type='message', role=<AuthorRole.USER: 'user'>, name=None, items=[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='test', encoding=None)], encoding=None, finish_reason=None, status=None)])